# 词语相似性和类比
:label:`sec_synonyms`

在 :numref:`sec_word2vec_pretraining` 中，  
我们在一个小数据集上训练了一个 word2vec 模型，并将其应用于找到输入词的语义相似词。实际上，可以在大型语料库上预训练的词向量可以应用于下游的自然语言处理任务，这将在 :numref:`chap_nlp_app` 中介绍。为了直观地展示从大型语料库中预训练的词向量的语义，让我们将它们应用于词语相似性和类比任务。

In [1]:
import os
import torch
from torch import nn
from d2l import torch as d2l

## 加载预训练词向量

下面列出了维度为50、100和300的预训练GloVe嵌入，可以从[GloVe网站](https://nlp.stanford.edu/projects/glove/)下载。
预训练的fastText嵌入有多种语言版本。这里我们考虑一个英语版本（300维的"wiki.en"），可以从[fastText网站](https://fasttext.cc/)下载。

In [2]:
#@save
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')

#@save
d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')

#@save
d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')

#@save
d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')

为了加载这些预训练的GloVe和fastText嵌入，我们定义了以下`TokenEmbedding`类。

In [3]:
#@save
class TokenEmbedding:
    """Token Embedding."""
    def __init__(self, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(
            embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {token: idx for idx, token in
                             enumerate(self.idx_to_token)}

    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe website: https://nlp.stanford.edu/projects/glove/
        # fastText website: https://fasttext.cc/
        with open(os.path.join(data_dir, 'vec.txt'), 'r') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # Skip header information, such as the top row in fastText
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [self.token_to_idx.get(token, self.unknown_idx)
                   for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

在下面的操作中，我们加载了
50维的GloVe嵌入
（在维基百科子集上预训练）。
在创建`TokenEmbedding`实例时，
如果指定的嵌入文件尚未下载，则需要下载。

In [4]:
glove_6b50d = TokenEmbedding('glove.6b.50d')

输出词汇量。词汇表包含400000个词（词元）和一个特殊的未知词元。

In [5]:
len(glove_6b50d)

400001

我们可以得到词汇表中单词的索引，反之亦然。

In [ ]:
glove_6b50d.token_to_idx['beautiful'], glove_6b50d.idx_to_token[3367]

(3367, 'beautiful')

## 应用预训练词向量

使用加载的GloVe向量，
我们将通过以下词语相似性和类比任务
来展示它们的语义。


### 词语相似性

类似于 :numref:`subsec_apply-word-embed`，
为了根据词向量之间的余弦相似度
找到输入词的语义相似词，
我们实现了以下 `knn`
（$k$-最近邻）函数。

In [7]:
def knn(W, x, k):
    # Add 1e-9 for numerical stability
    cos = torch.mv(W, x.reshape(-1,)) / (
        torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) *
        torch.sqrt((x * x).sum()))
    _, topk = torch.topk(cos, k=k)
    return topk, [cos[int(i)] for i in topk]

然后，我们
使用从`TokenEmbedding`实例`embed`中的预训练词向量
来搜索相似的词。

In [8]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):  # Exclude the input word
        print(f'cosine sim={float(c):.3f}: {embed.idx_to_token[int(i)]}')

`glove_6b50d`的预训练词向量词汇包含400000个单词和一个特殊的未知标记。除了输入的单词和未知标记外，在这个词汇表中，让我们找到与单词"chip"语义最相似的三个单词。

In [9]:
get_similar_tokens('chip', 3, glove_6b50d)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics


Below outputs similar words
to "baby" and "美丽".

In [10]:
get_similar_tokens('baby', 3, glove_6b50d)

cosine sim=0.839: babies
cosine sim=0.800: boy
cosine sim=0.792: girl


In [11]:
get_similar_tokens('beautiful', 3, glove_6b50d)

cosine sim=0.921: lovely
cosine sim=0.893: gorgeous
cosine sim=0.830: wonderful


### 词类比

除了寻找相似的词，
我们还可以将词向量
应用于词类比任务。
例如，
“man”：“woman”::“son”：“daughter”
是词类比的形式：
“man”与“woman”的关系就像“son”与“daughter”的关系。
具体来说，
词类比完成任务
可以定义为：
对于词类比
$a : b :: c : d$，给定前三个词 $a$、$b$ 和 $c$，找到 $d$。
用 $\textrm{vec}(w)$ 表示词 $w$ 的向量。
为了完成类比，
我们将找到其向量
与 $\textrm{vec}(c)+\textrm{vec}(b)-\textrm{vec}(a)$ 结果最相似的词。

In [12]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed[[token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[int(topk[0])]  # Remove unknown words

让我们使用加载的词向量来验证“男性-女性”类比。

In [13]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

下面完成了一个“首都-国家”类比：
“beijing”：“china”::“tokyo”：“japan”。
这展示了预训练词向量中的语义。

In [14]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'japan'

对于
“形容词-最高级形容词”类比
例如
“bad”:“worst”::“big”:“biggest”，
我们可以看到预训练的词向量
可能捕捉到了语法信息。

In [15]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'

为了展示预训练词向量中捕捉到的过去时的概念，我们可以使用“现在时-过去时”的类比进行测试：“do”：“did” :: “go”：“went”。

In [16]:
get_analogy('do', 'did', 'go', glove_6b50d)

'went'

## 摘要

* 在实践中，可以在大型语料库上预训练的词向量可以应用于下游自然语言处理任务。
* 预训练的词向量可以应用于词相似性和类比任务。

## 练习

1. 使用 `TokenEmbedding('wiki.en')` 测试 fastText 的结果。
1. 当词汇量非常大时，如何更快地找到相似词或完成词类比？

[讨论](https://discuss.d2l.ai/t/1336)